# Pin study: comparing CE and multigroup results within OpenMC

## Import modules

In [25]:
import sys
import os

import openmc
import openmc.mgxs as mgxs

import numpy as np
import matplotlib.pyplot as plt
import copy as copy

import pandas as pd
from IPython.display import Image
import glob

## Add location of OpenMC XS executable, setup ENDF xs path

In [26]:
import os

# Add path to OpenMC binary
# os.environ['PATH'] += r':/path/to/openmc/bin'
os.environ['PATH'] += r':/Users/jean.ragusa/repo/openmc/local/bin'

# Add location of OpenMC xs data
%env OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/Users/jean.ragusa/repo/endfb-viii.0-hdf5/cross_sections.xml


## Cleanup .xml files 

In [27]:
clean_file_list = ["model.xml", "materials.xml", "geometry.xml","plots.xml","geometry.xml",\
                   "settings.xml","tallies.out","summary.h5","statepoint.50.h5"]

for file in clean_file_list :
    path_file = os.path.join(os.getcwd(), file)
    if os.path.exists(path_file):
        os.remove(path_file)

## Start model

In [28]:
model=openmc.Model()

### Define Materials

In [29]:
uo2 = openmc.Material(name='uo2')

uo2.add_nuclide('U234', 6.11864E-06, 'ao')
uo2.add_nuclide('U235', 7.18132E-04, 'ao')
uo2.add_nuclide('U236', 3.29861E-06, 'ao')
uo2.add_nuclide('U238', 2.21546E-02, 'ao')
uo2.add_nuclide('O16', 4.57642E-02, 'ao')

uo2.set_density('g/cm3', 10.257 )

In [30]:
zirconium = openmc.Material(name="zirconium")

zirconium.add_nuclide('Zr90', 2.18865E-02, 'ao')
zirconium.add_nuclide('Zr91', 4.77292E-03, 'ao')
zirconium.add_nuclide('Zr92', 7.29551E-03, 'ao')
zirconium.add_nuclide('Zr94', 7.39335E-03, 'ao')
zirconium.add_nuclide('Zr96', 1.19110E-03, 'ao')
zirconium.add_nuclide('Sn112',4.68066E-06, 'ao')
zirconium.add_nuclide('Sn114', 3.18478E-06, 'ao')
zirconium.add_nuclide('Sn115', 1.64064E-06, 'ao')
zirconium.add_nuclide('Sn116', 7.01616E-05, 'ao')
zirconium.add_nuclide('Sn117', 3.70592E-05, 'ao')
zirconium.add_nuclide('Sn118', 1.16872E-04, 'ao')
zirconium.add_nuclide('Sn119', 4.14504E-05, 'ao')
zirconium.add_nuclide('Sn120', 1.57212E-04, 'ao')
zirconium.add_nuclide('Sn122', 2.23417E-05, 'ao')
zirconium.add_nuclide('Sn124', 2.79392E-05, 'ao')
zirconium.add_nuclide('Fe54', 8.68307E-06, 'ao')
zirconium.add_nuclide('Fe56', 1.36306E-04, 'ao')
zirconium.add_nuclide('Fe57', 3.14789E-06, 'ao')
zirconium.add_nuclide('Fe58', 4.18926E-07, 'ao')
zirconium.add_nuclide('Cr50', 3.30121E-06, 'ao')
zirconium.add_nuclide('Cr52', 6.36606E-05, 'ao')
zirconium.add_nuclide('Cr53', 7.21860E-06, 'ao')
zirconium.add_nuclide('Cr54', 1.79686E-06, 'ao')
zirconium.add_nuclide('Hf174', 3.54138E-09, 'ao')
zirconium.add_nuclide('Hf176', 1.16423E-07, 'ao')
zirconium.add_nuclide('Hf177', 4.11686E-07, 'ao')
zirconium.add_nuclide('Hf178', 6.03806E-07, 'ao')
zirconium.add_nuclide('Hf179', 3.01460E-07, 'ao')
zirconium.add_nuclide('Hf180', 7.76449E-07, 'ao')

zirconium.set_density('g/cm3',  6.56)

In [31]:
water = openmc.Material(name="water")

water.add_nuclide('H1',4.96224E-02, 'ao')
water.add_nuclide('O16', 2.48112E-02, 'ao')
water.add_nuclide('B10', 1.07070E-05, 'ao')
water.add_nuclide('B11', 4.30971E-05, 'ao')

water.set_density('g/cm3', 0.743)

water.add_s_alpha_beta('c_H_in_H2O')

In [32]:
# materials_file = openmc.Materials([uo2, zirconium, water]) 

model.materials = openmc.Materials([uo2, zirconium, water]) 
print(model.materials)

[Material
	ID             =	19
	Name           =	uo2
	Temperature    =	None
	Density        =	10.257 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	True
	S(a,b) Tables  
	Nuclides       
	U234           =	6.11864e-06  [ao]
	U235           =	0.000718132  [ao]
	U236           =	3.29861e-06  [ao]
	U238           =	0.0221546    [ao]
	O16            =	0.0457642    [ao]
, Material
	ID             =	20
	Name           =	zirconium
	Temperature    =	None
	Density        =	6.56 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	False
	S(a,b) Tables  
	Nuclides       
	Zr90           =	0.0218865    [ao]
	Zr91           =	0.00477292   [ao]
	Zr92           =	0.00729551   [ao]
	Zr94           =	0.00739335   [ao]
	Zr96           =	0.0011911    [ao]
	Sn112          =	4.68066e-06  [ao]
	Sn114          =	3.18478e-06  [ao]
	Sn115          =	1.64064e-06  [ao]
	Sn116          =	7.01616e-05  [ao]
	Sn117          =	3.70592e-05  [ao]
	Sn118          =	0.000116872  [ao]
	Sn119          =	4.14504e

### Create pincell: fuel, clad, water (no gap)

In [33]:
pitch = 1.26
rod_height = 1.

fuel_outer_radius = openmc.ZCylinder(r=0.418)
clad_outer_radius = openmc.ZCylinder(r=0.475)

left   = openmc.XPlane(-pitch/2, boundary_type='reflective')
right  = openmc.XPlane( pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top    = openmc.YPlane( pitch/2, boundary_type='reflective')

In [34]:
fuel_region  = -fuel_outer_radius
clad_region  = +fuel_outer_radius & -clad_outer_radius
water_region = +left & -right & +bottom & -top & +clad_outer_radius

In [35]:
fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region
fuel.temperature = 565

In [36]:
clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region
clad.temperature = 565

In [37]:
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region 
moderator.temperature = 565

In [38]:
root_universe = openmc.Universe(cells=(fuel, clad, moderator))
model.geometry = openmc.Geometry(root_universe)

### Plot

model.materials.export_to_xml()
model.geometry.export_to_xml()

plots = []

plot = openmc.Plot.from_geometry(model.geometry)
plot.pixels = (500, 500)
dr = pitch * 0.05
plot.width = (pitch+dr, pitch+dr)
plot.origin = (0., 0., 0)
plot.color_by = 'cell'

plots.append(plot)
model.plots=openmc.Plots(plots)
plot.to_ipython_image(openmc_exec='openmc', cwd='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/vera')

###  Settings

In [39]:
settings = openmc.Settings()

bounds = [-pitch/2, -pitch/2, -rod_height/2, pitch/2, +pitch/2, rod_height/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)

source = openmc.IndependentSource(space=uniform_dist)

settings.source = source

settings.batches = 30
settings.inactive = 20
settings.particles = 20000
settings.keff_trigger = {'type':'std_dev','threshold':0.00030}
settings.trigger_active = True
settings.trigger_max_batches = 50000
settings.output = {'tallies':True}

settings.temperature['method'] = 'interpolation'

model.settings = settings

settings_file = settings

### Tallies

In [40]:
tallies = openmc.Tallies()
model.tallies = tallies

### Define and choose energy group

In [41]:
egroup_name = 'XMAS-172'
egroup = openmc.mgxs.GROUP_STRUCTURES[egroup_name]

### Create MGXS tallies for mgxs directory

In [42]:
mgxs_lib_dict = {}

pn_order = [0,1,3,5,7]

for pn in pn_order:
    
    mgxs_lib_ = openmc.mgxs.Library(model.geometry)
    groups = openmc.mgxs.EnergyGroups(egroup)

    mgxs_lib_.energy_groups = groups
    mgxs_lib_.scatter_format = "legendre"
    mgxs_lib_.mgxs_types = ['transport','nu-transport','absorption', 'nu-fission', 'fission' ,\
                                  'chi', 'consistent scatter matrix', 'multiplicity matrix','total',\
                                  'consistent nu-scatter matrix']

    mgxs_lib_.legendre_order = pn
    if pn == 0:
        mgxs_lib_.correction = None
        
    mgxs_lib_.by_nuclide = False
    mgxs_lib_.domain_type = 'cell'
    mgxs_lib_.domains = model.geometry.get_all_material_cells().values()
    mgxs_lib_.build_library()
    mgxs_lib_.check_library_for_openmc_mgxs()
    
    mgxs_lib_.add_to_tallies_file(tallies, merge=False)
    
    mgxs_lib_dict[pn] = copy.deepcopy(mgxs_lib_)

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 1 is greater than zero
  warn(msg, RuntimeWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 5 is greater than zero
  warn(msg, RuntimeWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 7 is greater than zero
  warn(msg, RuntimeWarning)


### Create power tally

model.export_to_model_xml('model_iso.xml')

### Run OpenMC model

In [43]:
statepoint_filename = model.run()

/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=451.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=452.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=453.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=454.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=471.
  warn(msg, IDWarning)
/Users/jean.ragusa/anaconda3/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=481.
  warn(msg, IDWarning)
/Users/jean.ragu

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

       43/1    1.19154    1.18846 +/- 0.00214
 Triggers unsatisfied, max unc./thresh. is 4.883251973278263 for eigenvalue
 The estimated number of batches is 569
       44/1    1.19059    1.18855 +/- 0.00205
 Triggers unsatisfied, max unc./thresh. is 4.696196416985734 for eigenvalue
 The estimated number of batches is 550
       45/1    1.17200    1.18789 +/- 0.00208
 Triggers unsatisfied, max unc./thresh. is 4.437113798974768 for eigenvalue
 The estimated number of batches is 513
       46/1    1.19117    1.18802 +/- 0.00200
 Triggers unsatisfied, max unc./thresh. is 4.279355214879578 for eigenvalue
 The estimated number of batches is 497
       47/1    1.19021    1.18810 +/- 0.00193
 Triggers unsatisfied, max unc./thresh. is 4.179002142621444 for eigenvalue
 The estimated number of batches is 492
       48/1    1.18768    1.18808 +/- 0.00186
 Triggers unsatisfied, max unc./thresh. is 4.0141385680555475 for eigenvalue
 The estimated number of batches is 472
       49/1    1.16901    1

       94/1    1.16330    1.18539 +/- 0.00132
 Triggers unsatisfied, max unc./thresh. is 2.8073118843756775 for eigenvalue
 The estimated number of batches is 604
       95/1    1.18356    1.18537 +/- 0.00131
 Triggers unsatisfied, max unc./thresh. is 2.774054482319444 for eigenvalue
 The estimated number of batches is 598
       96/1    1.18687    1.18539 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.7610729271154457 for eigenvalue
 The estimated number of batches is 600
       97/1    1.16369    1.18511 +/- 0.00130
 Triggers unsatisfied, max unc./thresh. is 2.8128926116731936 for eigenvalue
 The estimated number of batches is 630
       98/1    1.19096    1.18518 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.784865229344582 for eigenvalue
 The estimated number of batches is 625
       99/1    1.20209    1.18540 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.7626094846021974 for eigenvalue
 The estimated number of batches is 623
      100/1    1.18019  

      145/1    1.19259    1.18583 +/- 0.00098
 Triggers unsatisfied, max unc./thresh. is 1.914429781031651 for eigenvalue
 The estimated number of batches is 479
      146/1    1.17604    1.18575 +/- 0.00097
 Triggers unsatisfied, max unc./thresh. is 1.9103551534798793 for eigenvalue
 The estimated number of batches is 480
      147/1    1.18438    1.18574 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.9006047609558998 for eigenvalue
 The estimated number of batches is 479
      148/1    1.19933    1.18585 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.8915113449394862 for eigenvalue
 The estimated number of batches is 478
      149/1    1.17377    1.18575 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.8846483133966088 for eigenvalue
 The estimated number of batches is 479
      150/1    1.20365    1.18589 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.874861442897387 for eigenvalue
 The estimated number of batches is 477
      151/1    1.18998  

...

      247/1    1.18463    1.18541 +/- 0.00073
 Triggers unsatisfied, max unc./thresh. is 1.3465935875717328 for eigenvalue
 The estimated number of batches is 432
      248/1    1.18636    1.18541 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3458468803633066 for eigenvalue
 The estimated number of batches is 433
      249/1    1.18592    1.18541 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.343999801584996 for eigenvalue
 The estimated number of batches is 434
      250/1    1.17765    1.18538 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.341493186375959 for eigenvalue
 The estimated number of batches is 434
      251/1    1.18179    1.18536 +/- 0.00072
 Triggers unsatisfied, max unc./thresh. is 1.3366924835195209 for eigenvalue
 The estimated number of batches is 433
      252/1    1.17693    1.18533 +/- 0.00071
 Triggers unsatisfied, max unc./thresh. is 1.33127707162312 for eigenvalue
 The estimated number of batches is 432
      253/1    1.19146    

      298/1    1.20251    1.18547 +/- 0.00064
 Triggers unsatisfied, max unc./thresh. is 1.2276628841420376 for eigenvalue
 The estimated number of batches is 439
      299/1    1.19134    1.18549 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2243871874341432 for eigenvalue
 The estimated number of batches is 439
      300/1    1.20036    1.18555 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.219662383932401 for eigenvalue
 The estimated number of batches is 437
      301/1    1.19266    1.18557 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2157910299266474 for eigenvalue
 The estimated number of batches is 436
      302/1    1.18806    1.18558 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.2119575259144966 for eigenvalue
 The estimated number of batches is 435
      303/1    1.18239    1.18557 +/- 0.00063
 Triggers unsatisfied, max unc./thresh. is 1.207761023100481 for eigenvalue
 The estimated number of batches is 433
      304/1    1.20221  

      349/1    1.20106    1.18565 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.155791426549743 for eigenvalue
 The estimated number of batches is 460
      350/1    1.18224    1.18564 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.152322999121656 for eigenvalue
 The estimated number of batches is 459
      351/1    1.19508    1.18567 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1494568273114392 for eigenvalue
 The estimated number of batches is 458
      352/1    1.18897    1.18568 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1469097747795856 for eigenvalue
 The estimated number of batches is 457
      353/1    1.20134    1.18573 +/- 0.00059
 Triggers unsatisfied, max unc./thresh. is 1.1449815741277467 for eigenvalue
 The estimated number of batches is 457
      354/1    1.18982    1.18574 +/- 0.00058
 Triggers unsatisfied, max unc./thresh. is 1.1420985035973568 for eigenvalue
 The estimated number of batches is 456
      355/1    1.16650  

      400/1    1.19875    1.18598 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0755127555456434 for eigenvalue
 The estimated number of batches is 460
      401/1    1.19151    1.18600 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0732042386008598 for eigenvalue
 The estimated number of batches is 459
      402/1    1.20354    1.18604 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0714451733112667 for eigenvalue
 The estimated number of batches is 459
      403/1    1.19059    1.18606 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0688177022611547 for eigenvalue
 The estimated number of batches is 458
      404/1    1.18731    1.18606 +/- 0.00055
 Triggers unsatisfied, max unc./thresh. is 1.0663359693821541 for eigenvalue
 The estimated number of batches is 457
      405/1    1.18999    1.18607 +/- 0.00054
 Triggers unsatisfied, max unc./thresh. is 1.0642680738456016 for eigenvalue
 The estimated number of batches is 457
      406/1    1.19160

      451/1    1.18702    1.18663 +/- 0.00052
 Triggers satisfied for batch 451
 Creating state point statepoint.00451.h5...

 =======================>     TIMING STATISTICS     <=======================

 Total time for initialization     = 3.5883e+00 seconds
   Reading cross sections          = 3.5706e+00 seconds
 Total time in simulation          = 8.0578e+02 seconds
   Time in transport only          = 7.9225e+02 seconds
   Time in inactive batches        = 2.7071e+00 seconds
   Time in active batches          = 8.0307e+02 seconds
   Time synchronizing fission bank = 2.2588e-01 seconds
     Sampling source sites         = 1.9784e-01 seconds
     SEND/RECV source sites        = 2.7869e-02 seconds
   Time accumulating tallies       = 1.2779e+01 seconds
   Time writing statepoints        = 4.1919e-01 seconds
 Total time for finalization       = 4.1797e+00 seconds
 Total time elapsed                = 8.1359e+02 seconds
 Calculation Rate (inactive)       = 147758 particles/second
 Calcul

In [44]:
ce_spfile = './statepoint_ce.h5'
ce_sumfile = './summary_ce.h5'

In [45]:
os.rename(statepoint_filename, ce_spfile)
os.rename('summary.h5', ce_sumfile)

In [58]:
# trick to make several runs work with jupyter
try:
    sp 
    print('sp found')
    sp.close()
except NameError:
    print('sp NOT found')

sp found


In [59]:
sp = openmc.StatePoint(ce_spfile, autolink=False)
su = openmc.Summary(ce_sumfile)
sp.link_with_summary(su)

In [61]:
for pn in pn_order:
    print("pn=",pn)
    mgxs_lib_order = copy.deepcopy(mgxs_lib_dict[pn])
    
    mgxs_lib_order.load_from_statepoint(sp)
    
# this may not be necessary:
#     mgxs_file_order = mgxs_lib_order.create_mg_library(xs_type='macro', \
#                                                     xsdata_names=['fuel', 'clad', 'moderator'])
        
    mgxs_file_order, materials_file, geometry_file = mgxs_lib_order.create_mg_mode(\
                                                        xsdata_names=['fuel', 'clad', 'moderator'])

    mgxs_file_order.xsdatas[0].temperatures=[565.]
    mgxs_file_order.xsdatas[1].temperatures=[565.]
    mgxs_file_order.xsdatas[2].temperatures=[565.]

    mgxs_file_order.export_to_hdf5(f'mgxs_order{pn}.h5')
    materials_file.cross_sections = f'mgxs_order{pn}.h5'
    
    model_order = openmc.Model()
    model_order.materials = materials_file
    model_order.geometry = geometry_file
    settings_file.energy_mode = 'multi-group'
    model_order.settings = settings_file   
    
    statepoint_filename_order = model_order.run()

pn= 0
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 ###############

 Preparing distributed cell instances...
 Writing summary.h5 file...
 Initializing source particles...

 ====================>     K EIGENVALUE SIMULATION     <====================

  Bat./Gen.      k            Average k
  =========   ========   ====================
        1/1    1.16604
        2/1    1.19871


        3/1    1.19550
        4/1    1.16360
        5/1    1.18890
        6/1    1.19257
        7/1    1.18796
        8/1    1.18507
        9/1    1.18460
       10/1    1.18705
       11/1    1.18346
       12/1    1.17335
       13/1    1.16700
       14/1    1.18335
       15/1    1.17179
       16/1    1.18209
       17/1    1.18507
       18/1    1.17804
       19/1    1.16630
       20/1    1.17961
       21/1    1.18486
       22/1    1.18152    1.18319 +/- 0.00167
       23/1    1.18866    1.18501 +/- 0.00206
       24/1    1.18082    1.18397 +/- 0.00180
       25/1    1.17695    1.18256 +/- 0.00198
       26/1    1.18989    1.18378 +/- 0.00202
       27/1    1.18562    1.18405 +/- 0.00173
       28/1    1.17300    1.18266 +/- 0.00204
       29/1    1.16797    1.18103 +/- 0.00243
       30/1    1.16353    1.17928 +/- 0.00279
 Triggers unsatisfied, max unc./thresh. is 6.339186470647355 for eigenvalue
 The estimated number of batches is 422
 Creating state point statepoint.

       77/1    1.17724    1.17964 +/- 0.00134
 Triggers unsatisfied, max unc./thresh. is 2.590288997795135 for eigenvalue
 The estimated number of batches is 403
       78/1    1.17371    1.17954 +/- 0.00132
 Triggers unsatisfied, max unc./thresh. is 2.5489401494313 for eigenvalue
 The estimated number of batches is 397
       79/1    1.17906    1.17953 +/- 0.00130
 Triggers unsatisfied, max unc./thresh. is 2.510774143162756 for eigenvalue
 The estimated number of batches is 392
       80/1    1.18834    1.17968 +/- 0.00129
 Triggers unsatisfied, max unc./thresh. is 2.470997861371686 for eigenvalue
 The estimated number of batches is 387
       81/1    1.18732    1.17980 +/- 0.00127
 Triggers unsatisfied, max unc./thresh. is 2.484007322099119 for eigenvalue
 The estimated number of batches is 397
       82/1    1.18276    1.17985 +/- 0.00125
 Triggers unsatisfied, max unc./thresh. is 2.469168642000903 for eigenvalue
 The estimated number of batches is 399
       83/1    1.18448    1.17

      128/1    1.20364    1.18037 +/- 0.00099
 Triggers unsatisfied, max unc./thresh. is 1.8481937773251889 for eigenvalue
 The estimated number of batches is 389
      129/1    1.17490    1.18032 +/- 0.00099
 Triggers unsatisfied, max unc./thresh. is 1.8324575224240809 for eigenvalue
 The estimated number of batches is 387
      130/1    1.18144    1.18033 +/- 0.00098
 Triggers unsatisfied, max unc./thresh. is 1.8294144928796174 for eigenvalue
 The estimated number of batches is 389
      131/1    1.18707    1.18039 +/- 0.00097
 Triggers unsatisfied, max unc./thresh. is 1.8129932990360933 for eigenvalue
 The estimated number of batches is 385
      132/1    1.18536    1.18043 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.7968965598865025 for eigenvalue
 The estimated number of batches is 382
      133/1    1.19259    1.18054 +/- 0.00096
 Triggers unsatisfied, max unc./thresh. is 1.7950871518925786 for eigenvalue
 The estimated number of batches is 385
      134/1    1.19582

...

      179/1    1.18077    1.18193 +/- 0.00090
 Triggers unsatisfied, max unc./thresh. is 1.685018629495911 for eigenvalue
 The estimated number of batches is 472
      180/1    1.17837    1.18191 +/- 0.00090
 Triggers unsatisfied, max unc./thresh. is 1.679889384588402 for eigenvalue
 The estimated number of batches is 472
      181/1    1.17645    1.18188 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.6774940834715242 for eigenvalue
 The estimated number of batches is 474
      182/1    1.19039    1.18193 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.6680386579161048 for eigenvalue
 The estimated number of batches is 471
      183/1    1.16910    1.18185 +/- 0.00089
 Triggers unsatisfied, max unc./thresh. is 1.6603726502451595 for eigenvalue
 The estimated number of batches is 470
      184/1    1.17650    1.18182 +/- 0.00088
 Triggers unsatisfied, max unc./thresh. is 1.6530008080885217 for eigenvalue
 The estimated number of batches is 469
      185/1    1.16948  

      230/1    1.17850    1.18186 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.3949749486356995 for eigenvalue
 The estimated number of batches is 429
      231/1    1.17155    1.18181 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.3887785853687704 for eigenvalue
 The estimated number of batches is 427
      232/1    1.20269    1.18191 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.3896364759490556 for eigenvalue
 The estimated number of batches is 430
      233/1    1.20209    1.18200 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.3841778501573794 for eigenvalue
 The estimated number of batches is 429
      234/1    1.17178    1.18195 +/- 0.00077
 Triggers unsatisfied, max unc./thresh. is 1.3791182602128889 for eigenvalue
 The estimated number of batches is 428
      235/1    1.18615    1.18197 +/- 0.00076
 Triggers unsatisfied, max unc./thresh. is 1.3788723091664468 for eigenvalue
 The estimated number of batches is 429
      236/1    1.16653

      284/1    1.18662    1.18207 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2451129398173075 for eigenvalue
 The estimated number of batches is 430
      285/1    1.17488    1.18204 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.241162937707355 for eigenvalue
 The estimated number of batches is 429
      286/1    1.17791    1.18202 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2364874145819387 for eigenvalue
 The estimated number of batches is 427
      287/1    1.17605    1.18200 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2320636744326767 for eigenvalue
 The estimated number of batches is 426
      288/1    1.20205    1.18207 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2306692148215586 for eigenvalue
 The estimated number of batches is 426
      289/1    1.18777    1.18210 +/- 0.00067
 Triggers unsatisfied, max unc./thresh. is 1.2287626194622872 for eigenvalue
 The estimated number of batches is 427
      290/1    1.18306 

      338/1    1.18555    1.18200 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.099005814005728 for eigenvalue
 The estimated number of batches is 405
      339/1    1.17371    1.18198 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0955789462578776 for eigenvalue
 The estimated number of batches is 403
      340/1    1.19848    1.18203 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0937328736331189 for eigenvalue
 The estimated number of batches is 403
      341/1    1.17442    1.18201 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.1022088806444728 for eigenvalue
 The estimated number of batches is 410
      342/1    1.15704    1.18193 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0994402546678896 for eigenvalue
 The estimated number of batches is 410
      343/1    1.18179    1.18193 +/- 0.00060
 Triggers unsatisfied, max unc./thresh. is 1.0960242336214658 for eigenvalue
 The estimated number of batches is 409
      344/1    1.18199 

      392/1    1.16702    1.18196 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0291609292129809 for eigenvalue
 The estimated number of batches is 415
      393/1    1.17119    1.18193 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0292203420813952 for eigenvalue
 The estimated number of batches is 416
      394/1    1.17405    1.18191 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0262912242826252 for eigenvalue
 The estimated number of batches is 414
      395/1    1.17906    1.18191 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0236784696531545 for eigenvalue
 The estimated number of batches is 413
      396/1    1.16344    1.18186 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0269391948098954 for eigenvalue
 The estimated number of batches is 417
      397/1    1.18299    1.18186 +/- 0.00056
 Triggers unsatisfied, max unc./thresh. is 1.0248195456426277 for eigenvalue
 The estimated number of batches is 416
      398/1    1.19666